In [1]:
print("Hello World!")

Hello World!


In [2]:
import os
import subprocess
import time
import math
import sys
import torch
import numpy as np
from scipy.ndimage import gaussian_filter
import matplotlib
matplotlib.use("Agg")  # Use non-interactive backend
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # Ensure 3D plotting is available
import matplotlib.cm as cm
import matplotlib.colors as colors
import io
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.auto import tqdm
from IPython.display import Video, display

# Use the Solarized Light Two style for a pleasing look.
plt.style.use('Solarize_Light2')

# === Configuration ===
t_min, t_max = 0, 1
render_fps = 2048       # Rendering frames per second (for smooth slow motion)
output_fps = 30        # Final video playback FPS
n_qubits = 10           # Number of qubits
system_dim = 2 ** n_qubits         # Hilbert space dimension
s = int(np.sqrt(system_dim))         # Grid size (assumed square)
height_scale = 1
sigma = 1

# Figure settings
fig_size = (10, 10)   # Figure size in inches for rendering
dpi = 150             # Resolution for saving figures (dots per inch)
color_map = 'hsv'     # Color mapping for phase (overrides the theme)
num_frames = int(math.ceil(render_fps * (t_max - t_min)))
# ======================

# Set up device (using MPS if available, else CPU)
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# --- Quantum Gates Setup ---
I = torch.eye(2, dtype=torch.complex64, device=device)
X = torch.tensor([[0, 1], [1, 0]], dtype=torch.complex64, device=device)
Y = torch.tensor([[0, -1j], [1j, 0]], dtype=torch.complex64, device=device)
Z = torch.tensor([[1, 0], [0, -1]], dtype=torch.complex64, device=device)

def tensor_product(operators):
    result = operators[0]
    for op in operators[1:]:
        result = torch.kron(result, op)
    return result

# --- Hamiltonian Construction ---
def build_hamiltonian():
    pauli_gates = [I, X, Y, Z]
    num_terms = 10
    H = torch.zeros((system_dim, system_dim), dtype=torch.complex64, device=device)
    for _ in range(num_terms):
        gate_indices = torch.randint(0, 4, (n_qubits,))
        paulis = [pauli_gates[i] for i in gate_indices]
        H += torch.randn(1, device=device) * tensor_product(paulis)
    return (H + H.mH) / 2  # Ensure Hermiticity

# --- Quantum System Initialization ---
print(f"Initializing {system_dim}-dim quantum system on {device}...")
H = build_hamiltonian()

# Create and normalize initial state vector
V_rand = (torch.randn(system_dim, dtype=torch.complex64, device=device) +
          1j * torch.randn(system_dim, dtype=torch.complex64, device=device))
norm = torch.sqrt(torch.sum(torch.abs(V_rand) ** 2))
V_rand /= norm

# --- Precompute Evolution ---
print("Precomputing time evolution...")
pre_start = time.time()

# Compute eigen-decomposition (temporarily on CPU in double precision)
H_cpu = H.cpu().to(torch.complex128)
eigvals, Q = torch.linalg.eigh(H_cpu)

# Convert eigenvalues and Q back to appropriate types and move to device.
eigvals = eigvals.to(device, dtype=torch.float32)
Q = Q.to(device, dtype=H.dtype)

t_values = torch.linspace(t_min, t_max, num_frames, device=device)
Z_all = torch.empty((s, s, num_frames), device=device)
C_all = torch.empty((s, s, num_frames), device=device)

for i, t in enumerate(t_values):
    sys.stdout.write(f"\rProcessing t = {t:.2f} ({i+1}/{num_frames})")
    sys.stdout.flush()
    phase_factors = torch.exp(-1j * eigvals * t)
    evolved = Q @ (phase_factors * (Q.mH @ V_rand))
    
    # Compute probability distribution and reshape as grid
    prob_data = torch.abs(evolved) ** 2
    prob_matrix = prob_data.view(s, s)
    filtered_np = gaussian_filter(prob_matrix.cpu().numpy() * height_scale, sigma=sigma)
    filtered = torch.tensor(filtered_np, device=device)
    fmin, fmax = filtered.min(), filtered.max()
    Z_all[:, :, i] = (filtered - fmin) / (fmax - fmin) if fmax > fmin else 0
    
    # Compute phase (normalized to [0, 1])
    evolved_view = evolved.view(s, s)
    real = evolved_view.real
    imag = evolved_view.imag
    phase = (torch.atan2(imag, real) + math.pi) / (2 * math.pi)
    phase_np = gaussian_filter(phase.cpu().numpy(), sigma=sigma) % 1
    C_all[:, :, i] = torch.tensor(phase_np, device=device)

# Move final results to CPU for plotting.
Z_all = Z_all.cpu().numpy()
C_all = C_all.cpu().numpy()
print(f"\nPrecomputed {num_frames} frames in {time.time()-pre_start:.2f}s")

# --- FFmpeg Setup ---
def create_ffmpeg_pipe(output_filename):
    return subprocess.Popen([
        'ffmpeg', '-y', '-loglevel', 'error',
        '-f', 'image2pipe',
        '-framerate', str(render_fps),
        '-c:v', 'png',
        '-i', '-',
        '-vf', f'pad=ceil(iw/2)*2:ceil(ih/2)*2,setpts=({render_fps}/{output_fps})*PTS',
        '-c:v', 'libx264',
        '-crf', '18',
        '-preset', 'slow',
        '-tune', 'animation',
        '-r', str(output_fps),
        '-pix_fmt', 'yuv420p',
        '-movflags', '+faststart',
        output_filename
    ], stdin=subprocess.PIPE)

output_filename = f"quantum_q{n_qubits}_render{render_fps}_output{output_fps}.mp4"
ffmpeg = create_ffmpeg_pipe(output_filename)

# Precompute the meshgrid for plotting.
X, Y = np.meshgrid(np.arange(s), np.arange(s))

# --- Define Frame Rendering Function ---
def render_frame(frame):
    """
    Renders a single frame and returns a tuple (frame_index, image_data)
    """
    fig = plt.figure(figsize=fig_size)
    ax = fig.add_subplot(111, projection='3d')
    
    # Set orthographic projection if available.
    try:
        ax.set_proj_type('ortho')
    except Exception:
        pass
    ax.view_init(elev=55, azim=45)
    
    # Aesthetic enhancements.
    ax.set_xlabel('X axis', fontsize=14, labelpad=10)
    ax.set_ylabel('Y axis', fontsize=14, labelpad=10)
    ax.set_zlabel('Amplitude', fontsize=14, labelpad=10)
    ax.tick_params(axis='both', which='major', labelsize=12)
    ax.grid(True, linestyle='--', alpha=0.7)
    
    # Retrieve data for the current frame.
    z_data = Z_all[:, :, frame]
    phase_data = C_all[:, :, frame]
    
    # Map phase to colors using the configured colormap.
    cmap = plt.get_cmap(color_map)
    facecolors = cmap(phase_data)
    
    # Plot the 3D surface.
    ax.plot_surface(X, Y, z_data, facecolors=facecolors,
                    rstride=1, cstride=1, linewidth=0, antialiased=True)
    
    # Set a title with the current simulation time.
    t_val = t_values[frame].item()
    ax.set_title(f"Time Evolution: t = {t_val:.2f}", pad=20, fontsize=16)
    
    # Add a colorbar to indicate phase.
    norm = colors.Normalize(vmin=0, vmax=1)
    sm = cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = fig.colorbar(sm, ax=ax, shrink=0.6, aspect=15, pad=0.1)
    cbar.set_label('Phase', fontsize=14)
    cbar.ax.tick_params(labelsize=12)
    
    # Save the figure to an in-memory buffer.
    buf = io.BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight', pad_inches=0.1, dpi=dpi)
    plt.close(fig)
    img_data = buf.getvalue()
    buf.close()
    return frame, img_data

# --- Multithreaded Rendering ---
# --- Multithreaded Rendering in Chunks ---
print("Rendering frames with multithreading in chunks...")
render_start = time.time()
results = [None] * num_frames

# Define the number of chunks (adjust as desired)
num_chunks = 4  
chunk_size = num_frames // num_chunks

# Create a list of (start, end) indices for each chunk,
# making sure to distribute any remainder frames.
chunks = []
start_idx = 0
for i in range(num_chunks):
    # Distribute remainder frames one per chunk until exhausted.
    extra = 1 if i < (num_frames % num_chunks) else 0
    end_idx = start_idx + chunk_size + extra
    chunks.append((start_idx, end_idx))
    start_idx = end_idx

def process_chunk(chunk_index, start, end):
    # Each chunk gets its own progress bar (using unique position)
    for frame in tqdm(range(start, end), desc=f"Chunk {chunk_index+1}/{num_chunks}", position=chunk_index, leave=False):
        frame_index, img_data = render_frame(frame)
        results[frame_index] = img_data

# Launch a separate thread for each chunk.
with ThreadPoolExecutor(max_workers=num_chunks) as executor:
    futures = [
        executor.submit(process_chunk, chunk_index, start, end)
        for chunk_index, (start, end) in enumerate(chunks)
    ]
    # Wait for all threads to complete.
    for future in as_completed(futures):
        future.result()

# Write rendered frames to FFmpeg in order.
for img_data in tqdm(results, desc="Writing frames to ffmpeg"):
    ffmpeg.stdin.write(img_data)

ffmpeg.stdin.close()
ffmpeg.wait()
total_render_time = time.time() - render_start
print(f"\nTotal render time: {total_render_time:.1f}s")
print(f"Output file: {output_filename}")


Initializing 1024-dim quantum system on mps...
Precomputing time evolution...
Processing t = 1.00 (2048/2048)
Precomputed 2048 frames in 16.35s
Rendering frames with multithreading in chunks...


Chunk 4/4:   0%|          | 0/512 [00:00<?, ?it/s]

Chunk 3/4:   0%|          | 0/512 [00:00<?, ?it/s]

Chunk 1/4:   0%|          | 0/512 [00:00<?, ?it/s]

Chunk 2/4:   0%|          | 0/512 [00:00<?, ?it/s]

Writing frames to ffmpeg:   0%|          | 0/2048 [00:00<?, ?it/s]


Total render time: 402.8s
Output file: quantum_q10_render2048_output30.mp4
